In [1]:
from reactome2py import content, analysis
import pprint 
import webbrowser
import itertools
import os

#### Pathway enrichment analysis of a patient's biomarkers 

In [2]:
markers = 'RAS,MAP,IL10,EGF,EGFR,STAT'
result = analysis.identifiers(ids=markers)
token = result['summary']['token']

on the go, we can use the analysis token to browse the result in reactome web portal

In [3]:
%%capture
url = 'https://reactome.org/PathwayBrowser/#/DTAB=AN&ANALYSIS=' + token
webbrowser.open(url)

or generate a full report to be reviewed later in time 

In [4]:
 if not os.path.exists('full-report/'):
    os.makedirs('full-report/')
    
analysis.report(token, path='full-report/', file='report.pdf', number='25', resource='TOTAL', 
                diagram_profile='Modern', analysis_profile='Standard', fireworks_profile='Barium Lithium', 
                species='Homo sapiens', chunk_size=128)

or intwine the enrichment analsyis result _**features**_ inside an in-house pipeline. This would be more time-efficient and close to real world use-cases.

1 - Find all significant pathways (p < 0.05) ordered by False Discover Rate (FDR)

In [5]:
token_result = analysis.token(token, species='Homo sapiens', page_size='-1', page='-1', sort_by='ENTITIES_FDR', 
                              order='ASC', resource='TOTAL', p_value='0.05', include_disease=True, 
                              min_entities=None, max_entities=None)

we can take a look at the json(dict/list) structure to get a sense of how to parse through fetching information.

In [6]:
# pprint.pprint(token_result)

2 - Fetch the pathway stable identifiers (stIds) for generating downstream results to further investigate. 

In [6]:
pathways = token_result['pathways']
pathways_stId = [p['stId'] for p in pathways] 

3 - Save the fireworks for systems/molecular biologists or clinicians to review the overall scope of pathways highlighted.

In [7]:
if not os.path.exists('fireworks/'):
    os.makedirs('fireworks/')
    
content.export_fireworks(species='9606', ext='png', file='fireworks_report', path='fireworks/', quality='10', 
                         flag=None, flag_interactors=False, sel=[], title=True, margin='15', resource='Total', 
                         diagram_profile='', coverage=False, token=token, exp_column=None)

4 - Fetch downstream pathways stIds that have diagrams, and pass the enrichment analysis result's token to highlight the reactions involved. Save all diagrams for review and comparison.

In [8]:
%%capture
fetch_downstream_pathways = [content.pathways_low_diagram(id=stId, species=None, all_forms=False) for stId in pathways_stId]

downstream_pathways = [low_pathway for low_pathway in fetch_downstream_pathways if low_pathway is not None]
has_diagram = [p[0]['hasDiagram'] for p in downstream_pathways]
downstream_pathway_has_diagram = list(itertools.compress(downstream_pathways, has_diagram))
downstream_pathway_stId = [p[0]['stId'] for p in downstream_pathway_has_diagram]

if not os.path.exists('diagrams/'):
    os.makedirs('diagrams/')
    
[content.export_diagram(id=stId, ext='png', quality='5', flag_interactors=False, title=True, margin='15',
                        ehld=True, diagram_profile='Modern', resource='Total', analysis_profile='Standard', 
                        token=token, flag=None, sel=[], exp_column=None, file="-".join([stId,'report']), path='diagrams/') 
 for stId in downstream_pathway_stId]

NOTE: in all steps we passed the enrichment analysis result **token** to highlight the active reactions unique to this patient's state. 